
<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°1, V0.3 </h1>
    
</center>

<p>

</p>

## Instrucciones

* La tarea es individual, sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea.**
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_
* La fecha de entrega es el jueves 6 de mayo a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.

## Introducción 

En esta primera tarea de INF-285, versión 2021-1, estudiaremos la importancia de los primeros temas estudiados en el curso, los cuales son: Representación de Punto Flotante, Pérdida de Importancia, Errores de Cancelación y Búsqueda de Ceros. 
El desarrollo de cada uno de esos temas se presenta en una serie de preguntas donde deberá ir decidiendo, pregunta a pregunta, cómo cada uno de los temas se aplica.
En general, los temas no se analizan por separado, sino de manera acoplada. 
Es muy importante que cada uno de los problemas sea analizado teóricamente primero, para luego poner su conocimiento en acción. 
Cada problema puede ser desarrollado de diversas formas, sin embargo, es muy importante determinar al final si el camino elegido resuelve la pregunta presentada.
Es decir, debe aplicar la metodología IDEA!

## Problemas

### 1. Simulador (60 ptos) 

Dada la familia de polinomios de grado 3 con la forma:


\begin{equation}
f(x)=-1 + x + 2 A^2 x - 3 A^2 x^2 + A^2 x^3
\end{equation}

Se pide implementar un algoritmo que reciba como parámetros el valor de $A$ (con $|A|>>1$), 'bits_mant' que corresponde a la cantidad de bits que tiene la mantisa y 'bits_exp' que corresponde a la cantidad de bits que tiene el exponente. 
El algoritmo se ejecutará en un computador ficticio que representa los números con esa cantidad de bits para el estándar de punto flotante IEEE. 
Debe controlar los casos especiales correctamente.
Esta implementación debe calcular las raíces de $f$ sin pérdida de **importancia** con la cantidad de bits disponibles para la mantisa y la cantidad de bits para el exponente.
Para obtener las raíces de $f$ usted debe encontrar de forma algebraica sus raíces y luego proponer un algoritmo basado en las _fórmulas_ obtenidas.

El computador ficticio cuenta con las operaciones matemáticas necesarias para obtener las raíces. 
Considere como límites de bits para la mantisa en el intervalo $[2,52]$ y en el intervalo $[2,11]$ para los bits del exponente.

Ejemplo:

```python
# Algoritmo de ejemplo
a = 9819824.624837
b = 148736.523476
c = a+b

# Implementación de ejemplo con Representación de Punto Flotante de 'bits_mant' bits en la mantisa y 'bits_exp' bits en el exponente.
am = f_new_rep(9819824.624837,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
bm = f_new_rep(148736.523476,bits_mant,bits_exp) # Aproximar el input en la nueva representación.
cm = f_suma(am,bm,bits_mant,bits_exp) # Aproximar el output de la suma y cada operación en la nueva representación.
```


In [121]:
import bitstring as bs

def enteroABinario(numero):
    numero = str(numero)
    parte_entera = int(numero.split('.')[0])
    parte_entera_bin = ""
    if parte_entera == 0:
        return '0'
    while(parte_entera >= 1):
        if parte_entera % 2 == 0:
            parte_entera_bin += "0"
        else:
            parte_entera_bin += "1"
        parte_entera = int(parte_entera/2)
    return parte_entera_bin[::-1]

def parteMantisaADecimal(numero):
    parte_decimal = 0
    j = -1
    for bit in numero:
        if bit == '1':
            parte_decimal += 2**(j)
        j -=1
    return parte_decimal
        
def parteEnteraADecimal(numero):
    parte_entera = 0
    j = len(numero) - 1
    for bit in numero:
        if bit == '1':
            parte_entera += 2**j
        j -=1
    return parte_entera
        

def verCerosDespuesDeMantisa(mantisa,bits_mant):
    todosCeros = True
    bitsDespuesDelUltimo = mantisa[bits_mant+1:]
    for bit in bitsDespuesDelUltimo:
        if bit != '0':
            todosCeros = False
            break
    return todosCeros

def sumarUnoMantisa(mantisa,bits_mant):
    mantisa = mantisa[:bits_mant]
    temp = list(mantisa)
    length = len(temp) - 1
    
    while length > 0:
        if temp[length] == '0':
            temp[length] = '1'
            break
        else:
            temp[length] = '0'
            length = length - 1
    return ''.join(temp)


def reglaRedondeo(mantisa,bits_mant):
    if bits_mant == 52 :
        return mantisa
    
    if mantisa[bits_mant] == '0': #ej: bits_mant = 52, entonces revisará el bit numero 53, índice 52 
        return mantisa[:bits_mant]
    
    if mantisa[bits_mant] == '1':
        if verCerosDespuesDeMantisa(mantisa,bits_mant):
            if (mantisa[bits_mant-1]) == '1': #revisa el ultimo bit de la mantisa a redondear
                mantisa = sumarUnoMantisa(mantisa,bits_mant)
                return mantisa[:bits_mant]
            elif (mantisa[bits_mant-1]) == '0':
                return mantisa[:bits_mant]
        else:
            mantisa = sumarUnoMantisa(mantisa,bits_mant)
            return mantisa[:bits_mant]

    return mantisa[:bits_mant]


def arreglarExponente(exponente,bits_mant,bits_exp):
    rango_exp = range(1,(2**bits_exp)-1) #rango de 1 a 2^(bits_exp)
    sesgo_bits_exp = 2**(bits_exp-1) - 1
    rango_exp = [num - sesgo_bits_exp for num in rango_exp] #rango de p
#     print(rango_exp)
    if (exponente) in rango_exp: #si el exponente es representable en el rango, retorna el mismo exponente
        return exponente
    else:                        #si no es represantable, se busca el más cercano representable 
        if (exponente >= 1): 
            return rango_exp[-1]
        else:
            return rango_exp[0]

def puntoFlotanteADecimal(numero,bits_mant,exponente_p):
    #Casos exponente >, < y = con mantisa
    antes_de_exponente = numero.split('x')[0][1:]
#     print(numero)
#     print(antes_de_exponente)
    antes_de_exponente = antes_de_exponente[0:1] + antes_de_exponente[2::] #quitamos el . del punto flotante
#     print(antes_de_exponente)
    
    if exponente_p >= 0: #numeros que no están entre 0 y 1
        print("Exponente p > 0")
        if bits_mant == exponente_p:
            parte_entera = antes_de_exponente[:exponente_p+1]
            print("Bits mantisa == Exponente_p -> La parte entera decimal de "+ numero + " es " + str(parteEnteraADecimal(parte_entera)))
            return parteEnteraADecimal(parte_entera)

        elif bits_mant > exponente_p:
            parte_entera = antes_de_exponente[:exponente_p+1] #obtenemos la parte entera
            parte_decimal = antes_de_exponente[exponente_p+1:]
            print("Bits mantisa > Exponente_p -> La parte entera binaria de "+ numero + " es " + parte_entera)
            print("Bits mantisa > Exponente_p -> La parte decimal binaria de "+ numero + " es " + parte_decimal)
            print("Bits mantisa > Exponente_p -> La parte entera decimal de "+ numero + " es " + str(parteEnteraADecimal(parte_entera)))
            print("Bits mantisa > Exponente_p -> La parte decimal decimal de "+ numero + " es " + str(parteMantisaADecimal(parte_decimal)))
            return parteEnteraADecimal(parte_entera) + parteMantisaADecimal(parte_decimal)
        elif bits_mant < exponente_p:
            parte_entera = antes_de_exponente[:exponente_p+1]
            print("Bits mantisa < Exponente_p -> La parte entera decimal de "+ numero + " es " + str(parteEnteraADecimal(parte_entera)))
            return parteEnteraADecimal(parte_entera) * 2**(exponente_p-bits_mant-1)
    else: #falta numeros entre 1 y 0, preguntar caya
        print("Exponente p < 0")
        return(parteMantisaADecimal(antes_de_exponente) * 2**(exponente_p+1))


#Retorna el número binario en doble precisión (1 bit signo, 11 bits exponente, 52 mantisa)
def decimalAPuntoFlotante(f,bits_mant,bits_exp):
    b = bs.pack('>d', f)
    b = b.bin
    print(b[0]+' '+b[1:12]+ ' '+b[12:])
    #caso normal (no con exponentes límite)
    signo = '+' if b[0] == '0' else '-'
    exponente = int('0b'+b[1:12], 2)
    mantisa = b[12:]
    rango_exp = range(1,2047) #rango de 0 a 2^(bits_exp)
    sesgo = 2**(11-1) - 1
    exp_flotante = exponente - sesgo #2^exp_flotante
    numero = signo + '1.' + mantisa +' x 2^'+ str(exp_flotante)    
    print("----------------------------------------------------------------------------------------")
    print("   double precision: "+ signo + '1.' + mantisa +' x 2^'+ str(exp_flotante)) 
    print("----------------------------------------------------------------------------------------")

    #-------------Hasta acá, es todo en double precision----------------------#
                
    exp_flotante_arreglado = arreglarExponente(exp_flotante,bits_mant,bits_exp)
    print(exp_flotante)
    nuevo_numero = signo +'1.'+ reglaRedondeo(mantisa,bits_mant)+ ' x 2^'+ str(exp_flotante_arreglado)
    
    print("\n----------------------------------------------------------------------------------------")
    print("   con redondeo (mantisa y exp): "+ nuevo_numero)
    print("----------------------------------------------------------------------------------------") 
    
    retorno = puntoFlotanteADecimal(nuevo_numero,bits_mant,exp_flotante_arreglado)
    return retorno if signo == '+' else -1 * retorno

decimalAPuntoFlotante(0.234343423,15,6)
# 0.001110111111011011

0 01111111100 1101111111101111011100011100111010001010101000110100
----------------------------------------------------------------------------------------
   double precision: +1.1101111111101111011100011100111010001010101000110100 x 2^-3
----------------------------------------------------------------------------------------
-3

----------------------------------------------------------------------------------------
   con redondeo (mantisa y exp): +1.110111111111000 x 2^-3
----------------------------------------------------------------------------------------
Exponente p < 0


0.234344482421875

In [120]:
import numpy as np


"""
input
x         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_new_rep(x,bits_mant,bits_exp):
    b = bs.pack('>d', x)
    b = b.bin
    print(b[0]+' '+b[1:12]+ ' '+b[12:])
    #caso normal (no con exponentes límite)
    signo = '+' if b[0] == '0' else '-'
    exponente = int('0b'+b[1:12], 2)
    mantisa = b[12:]
    rango_exp = range(1,2047) #rango de 0 a 2^(bits_exp)
    sesgo = 2**(11-1) - 1
    exp_flotante = exponente - sesgo #2^exp_flotante
    numero = signo + '1.' + mantisa +' x 2^'+ str(exp_flotante)    
    print("----------------------------------------------------------------------------------------")
    print("   double precision: "+ signo + '1.' + mantisa +' x 2^'+ str(exp_flotante)) 
    print("----------------------------------------------------------------------------------------")

    #-------------Hasta acá, es todo en double precision----------------------#
                
    exp_flotante_arreglado = arreglarExponente(exp_flotante,bits_mant,bits_exp)
    print(exp_flotante_arreglado)
    nuevo_numero = signo +'1.'+ reglaRedondeo(mantisa,bits_mant)+ ' x 2^'+ str(exp_flotante_arreglado)
    
    print("\n----------------------------------------------------------------------------------------")
    print("   con redondeo (mantisa y exp): "+ nuevo_numero)
    print("----------------------------------------------------------------------------------------") 
    
    result = puntoFlotanteADecimal(nuevo_numero,bits_mant,exp_flotante_arreglado)
    return result if signo == '+' else -1 * result
    return  result

"""
input
n1        : (double) valor a sumar
n2        : (double) valor a sumar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_suma(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la suma de dos numeros manteniendo el estandar asignado
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_multiplicacion(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la multiplicación de dos numeros manteniendo el estandar asignado
    return result

"""
input
n1        : (double) valor a multiplicar
n2        : (double) valor a multiplicar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
result      : (double) resultado obtenido
"""
def f_division(n1,n2,bits_mant,bits_exp):
    #Algoritmo que calcula la división de dos numeros manteniendo el estandar asignado
    return result

"""
input
A         : (double) valor a evaluar
bits_mant : (int) cantidad de bits de la mantisa
bits_exp  : (int) cantidad de bits del exponente
output
x_roots      : (array) raices del polinomio ordenadas de menor a mayor
"""
def f_find_roots(A,bits_mant,bits_exp):
    # Algoritmo para encontrar raíces de f(x).
    return x_roots

f_new_rep(9819824.624837,52,11)

0 10000010110 0010101110101101011000010011111111101010101000101010
----------------------------------------------------------------------------------------
   double precision: +1.0010101110101101011000010011111111101010101000101010 x 2^23
----------------------------------------------------------------------------------------
[-1022, -1021, -1020, -1019, -1018, -1017, -1016, -1015, -1014, -1013, -1012, -1011, -1010, -1009, -1008, -1007, -1006, -1005, -1004, -1003, -1002, -1001, -1000, -999, -998, -997, -996, -995, -994, -993, -992, -991, -990, -989, -988, -987, -986, -985, -984, -983, -982, -981, -980, -979, -978, -977, -976, -975, -974, -973, -972, -971, -970, -969, -968, -967, -966, -965, -964, -963, -962, -961, -960, -959, -958, -957, -956, -955, -954, -953, -952, -951, -950, -949, -948, -947, -946, -945, -944, -943, -942, -941, -940, -939, -938, -937, -936, -935, -934, -933, -932, -931, -930, -929, -928, -927, -926, -925, -924, -923, -922, -921, -920, -919, -918, -917, -916, -915,

9819824.624837

### Sinusoidal de Oreman (40 puntos)
Considere la siguiente función en 3 variables:
\begin{equation}
T(t,\alpha,\beta) = \alpha\cdot \cos(t)+\beta\cdot \sin(\ln(t)).
\end{equation}

La cual permite calcular la temperatura de una cámara de ambiente dado los parámetros $\alpha$ y $\beta$ los cuales corresponden a las condiciones iniciales del experimento. 
Por otro lado $t$ representa el tiempo en días transcurridos desde que se inició.

A usted le permiten utilizar la cámara por un tiempo limitado, para esto planea terminar a los 25 días pero debe asegurarse que las sustancias sean extraídas a temperatura 0.

Cree un programa que reciba los parametros $\alpha$ y $\beta$ y que retorne el día en el cual deba ser extraído el material cumpliendo la condición de ser el día más cercano a 25 (ya sea antes o después).


## Desarrollo

Para poder calcular la raíz que esté más cercana a $t = 25$ para la Sinusoidal de Oreman, podemos ayudarnos utilizando el método de la bisección en dos intervalos distintos.

Como sabemos que la raíz que debe ser encontrada debe ser lo más cercana posible a $t = 25$ (ya sea para la izquierda o para la derecha) se tomarán dos intervalos distintos, que utilizarán $t = 25$ como una especie de *pivote*, por así decirlo. Entonces, el procedimiento funciona así:

* Tomamos dos intervalos distintos, uno para los números que podrían ser raíz que estén antes del $25$, y otro para los números que estén después del $25$

* __Intervalo 1__: $[Lim_{izq},25]$
* __Intervalo 2__: $[25,Lim_{der}]$

Donde $Lim_{izq} = Lim_{der} = 25$ inicialmente

* Luego, se aplicará Método de la Bisección a la función $T(t,\alpha,\beta)$ para cada intervalo por separado, en donde antes se chequeará que se cumpla la la restricción $T(Lim_{izq})T(25) < 0$ y $T(25)T(Lim_{der}) < 0$. Si lo anterior no se cumple, entonces se le restará o sumará a Lim-izq o Lim-der una tolerancia, en este caso $tol = 0.001$ según corresponda y volverá a intentar a realizar bisección, de esta manera para seguir ampliando nuestro intervalo hasta que se cumpla la propiedad $T(a)T(b) < 0$., y así hasta encontrar las raíces en $x_{izq}$ y $x_{der}$ los dos intervalos.

* Finalmente, como queremos obtener la raíz más cercana a $t = 25$, calculamos la diferencia en valor absoluto entre $t = 25$ y las ráices. El menor valor será la raíz elegida


In [67]:
import numpy as np

def bisect(f, a, b, tol=1e-10): #Función sacada del Jupyter del profe 03_roots_of_1D_equations
    fa = f(a)
    fb = f(b)
    i = 0
    # Just checking if the sign is not negative => not root  necessarily 
    if np.sign(f(a)*f(b)) >= 0:
#         print('f(a)f(b)<0 not satisfied!')
        return None
  
    #Printing the evolution of the computation of the root
    print(' i |     a     |     c     |     b     |     fa    |     fc     |     fb     |   b-a')
    print('----------------------------------------------------------------------------------------')
    
    while(b-a)/2 > tol:
        c = (a+b)/2.
        fc = f(c)
        print('%2d | %.7f | %.7f | %.7f | %.7f | %.7f | %.7f | %.7f' %
              (i+1, a, c, b, fa, fc, fb, b-a))
        # Did we find the root?
        if fc == 0:
            print('f(c)==0')
            break
        elif np.sign(fa*fc) < 0:
            b = c
            fb = fc
        else:
            a = c
            fa = fc
        i += 1
        
    xc = (a+b)/2.
    return xc

In [ ]:
"""
input
alpha: (float) parametro alpha
beta: (float) parametro beta
tol: (float) tolerancia
output
root: (double) raiz obtenida
"""
def raiz_de_oreman(alpha, beta, tol = 1e-10):
    T = lambda t: alpha*np.cos(t) + beta*np.sin(np.log(t))

    lim_izq = 25
    lim_der = 25  
    
    tol_rango = 1e-2
    x1 = bisect(T,lim_izq,25,tol=1e-10) 
    while(x1 == None): #avanzamos por la izquierda hasta encontrar la raíz
        lim_izq -= tol_rango
        x1 = bisect(T,lim_izq,25,tol=1e-10)
        
    x2 = bisect(T,25,lim_der,tol=1e-10)
    while(x2 == None): #avanzamos por la derecha hasta encontrar la raíz
        lim_der += tol_rango
        x2 = bisect(T,25,lim_der,tol=1e-10)

    diff_x1 = abs(25-x1)
    diff_x2 = abs(25-x2)

    root = x1 if diff_x1 < diff_x2 else x2
    return root

# Referencias

Si corresponde